In [ ]:
import sqlite3
import json

In [25]:
conn = sqlite3.connect('../databases/CovidData.db')
cursor = conn.cursor()

In [ ]:
create_research_table_sql = """
    CREATE TABLE IF NOT EXISTS covid_research (
        id TEXT PRIMARY KEY,
        source TEXT,
        pub_date TEXT,
        pub_timestamp TEXT,
        title TEXT,
        link TEXT,
        authors TEXT,
        authors_affiliations TEXT
    )
"""

create_keywords_table_sql = """
    CREATE TABLE IF NOT EXISTS covid_research_keywords (
        keyword TEXT PRIMARY KEY,
        count INTEGER
    )
"""

cursor.execute(create_research_table_sql)
cursor.execute(create_keywords_table_sql)

In [ ]:
from datetime import datetime
import csv

list_delimiter = ', '

with open('../data/combined_sources.json', 'r') as json_file:
    data = json.load(json_file)

    for item in data:
        if item['source'] == 'Doaj':
            # format journal date
            datetime_object = datetime.strptime(
                item['date'], '%Y-%m-%dT%H:%M:%SZ')
            datetime_timestamp = datetime.timestamp(datetime_object)

            # format authors and affiliations lists
            authors_list = []
            author_affiliations_list = []
            for author in item['authors']:
                authors_list.append(author.get('name', ""))
                author_affiliations_list.append(author.get('affiliation', ""))

            # format keywords lists
            with open('keyword_stopwords.csv', 'r') as file:
                reader = csv.reader(file, delimiter=',')
                for row in reader:
                    for keyword in item['keywords']:
                        if keyword.strip().lower().replace('-', '') not in row:
                            cursor.execute(
                                "INSERT INTO covid_research_keywords VALUES(?,?) ON CONFLICT(keyword) DO UPDATE SET count=count+1", (keyword, 1))

            cursor.execute("INSERT OR IGNORE INTO covid_research VALUES(?,?,?,?,?,?,?,?)", (
                item['id'],
                item['source'],
                str(datetime_object),
                str(datetime_timestamp),
                item['title'],
                list_delimiter.join([link['url'] for link in item['link']]),
                list_delimiter.join(authors_list),
                list_delimiter.join(author_affiliations_list),
            ))

        if item['source'] == 'PubMed':
            # format journal date
            formatted_date_str = f"{item['date']['Year']}-{item['date']['Month']}-{item['date']['Day']}"
            datetime_object = datetime.strptime(formatted_date_str, '%Y-%m-%d')
            datetime_timestamp = datetime.timestamp(datetime_object)

            # format authors and affiliations lists
            authors_list = []
            affiliations = []
            if isinstance(item['authors'], list):
                for author in item['authors']:
                    if author.get('AffiliationInfo'):
                        if isinstance(author['AffiliationInfo'], list):
                            for affiliation in author.get('AffiliationInfo', []):
                                affiliations.append(
                                    affiliation.get('Affiliation', ''))
                        elif isinstance(author['AffiliationInfo'], dict):
                            affiliations.append(author.get(
                                'AffiliationInfo', {}).get('Affiliation', ''))
                    authors_list.append(
                        f"{author.get('ForeName', '')} {author.get('LastName', '')}")

            # format keywords lists
            keywords_list = []
            for keyword in item.get('keywords', []):
                keywords_list.append(keyword['#text'])
                with open('keyword_stopwords.csv', 'r') as file:
                    reader = csv.reader(file, delimiter=',')
                    for row in reader:
                        if keyword['#text'].strip().lower().replace('-', '') not in row:
                            cursor.execute(
                                "INSERT INTO covid_research_keywords VALUES(?,?) ON CONFLICT(keyword) DO UPDATE SET count=count+1", (keyword['#text'], 1))

            title = item['title']
            if isinstance(item['title'], dict):
                title = item['title']['#text']

            # insert db row
            cursor.execute("INSERT OR IGNORE INTO covid_research VALUES(?,?,?,?,?,?,?,?)", (
                item['id'],
                item['source'],
                str(datetime_object),
                str(datetime_timestamp),
                title,
                item['link'],
                list_delimiter.join(authors_list),
                list_delimiter.join(affiliations),
            ))

In [30]:
# manually delete keywords

cursor.execute(
    """DELETE FROM covid_research_keywords WHERE keyword IN (
        'Pandemic', 'pandemic', 'COVID-19', 'SARS-CoV-2', 'covid-19', 'Pandemics', '“COVID-19”', 'covid-19'
        )""")

In [31]:
conn.commit()
conn.close()